# Using DALI with JAX

This simple example shows how to train a neural network implemented in JAX with DALI pipelines. It builds on MNIST training example from JAX codebse that can be found [here](https://github.com/google/jax/blob/jax-v0.4.13/examples/mnist_classifier_fromscratch.py).

We will use MNIST in Caffe2 format from [DALI_extra](https://github.com/NVIDIA/DALI_extra).

In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1,2'

import jax

print([d.device_kind for d in jax.devices()])

['NVIDIA T400 4GB', 'NVIDIA T400 4GB']


In [2]:
import os

training_data_path = os.path.join(os.environ['DALI_EXTRA_PATH'], 'db/MNIST/training/')
validation_data_path = os.path.join(os.environ['DALI_EXTRA_PATH'], 'db/MNIST/testing/')

First step is to create a pipeline definition function that will later be used to create instances of DALI pipelines. It defines all steps of the preprocessing. In this simple example we have `fn.readers.caffe2` for reading data in Caffe2 format, `fn.decoders.image` for image decoding, `fn.crop_mirror_normalize` used to normalize the images and `fn.reshape` to adjust the shape of the output tensors. We also move the labels from the CPU to the GPU memory with `labels.gpu()` and apply one hot encoding to them for training with `fn.one_hot`.

This example focuses on how to use DALI pipeline with JAX. For more information on DALI pipeline look into [Getting started](../../getting_started.ipynb) and [pipeline documentation](../../../pipeline.rst)

In [3]:
from nvidia.dali import pipeline_def
import nvidia.dali.fn as fn
import nvidia.dali.types as types


batch_size = 200
image_size = 28
num_classes = 10


@pipeline_def(device_id=0, batch_size=batch_size, num_threads=4)
def mnist_pipeline(data_path, random_shuffle):
    jpegs, labels = fn.readers.caffe2(
        path=data_path,
        random_shuffle=random_shuffle,
        name="mnist_caffe2_reader")
    images = fn.decoders.image(
        jpegs, device='mixed', output_type=types.GRAY)
    images = fn.crop_mirror_normalize(
        images, dtype=types.FLOAT, std=[255.], output_layout="CHW")
    images = fn.reshape(images, shape=[image_size * image_size])

    labels = labels.gpu()
    
    if random_shuffle:
        labels = fn.one_hot(labels, num_classes=num_classes)

    return images, labels

Next step is to instantiate pipelines and build them. Building creates and initializes pipeline internals.

In [4]:
print('Creating pipelines')
training_pipeline = mnist_pipeline(data_path=training_data_path, random_shuffle=True)
validation_pipeline = mnist_pipeline(data_path=validation_data_path, random_shuffle=False)

print('Building pipelines')
training_pipeline.build()
validation_pipeline.build()

print(training_pipeline)
print(validation_pipeline)

Creating pipelines
Building pipelines


For DALI pipeline to work with JAX it needs to be wrapped with appropriate DALI iterator. To get the iterator compatible with JAX we need to import it from DALI JAX plugin. In addition to the pipeline we can pass the `output_map`, `reader_name` and `auto_reset` parameters to the iterator. 

**Here is a quick explnation of how these parameters work:**

 - `output_map`: iterators return a dictionary with outputs of the pipeline as its values. Keys in this dictionary are defined by `output_map`. For example, `labels` output returned from the DALI pipeline defined above will be accessible as `iterator_output['labels']`,
 - `reader_name`: setting this parameter introduces the notion of an epoch to our iterator. DALI pipeline itself is infinite, it will return the data indefinately, wrapping around the dataset. DALI readers (such as `fn.readers.caffe2` used in this example) have access to the information about the size of the dataset. If we want to pass this information to the iterator we need to point to the operator that should be queried for the dataset size. We do it by naming the operator (note `name="mnist_caffe2_reader"`) and passing the same name as the value for `reader_name` argument,
  - `auto_reset`: this argument controls the behaviour of the iterator after the end of an epoch. If set to `True` will automatically reset the state of the iterator and prepare it to start the next epoch.

In [5]:
from nvidia.dali.plugin import jax as dax


print('Creating iterators')
training_iterator = dax.DALIGenericIterator(
    training_pipeline,
    output_map=["images", "labels"],
    reader_name="mnist_caffe2_reader",
    auto_reset=True)

validation_iterator = dax.DALIGenericIterator(
    validation_pipeline,
    output_map=["images", "labels"],
    reader_name="mnist_caffe2_reader",
    auto_reset=True)

print(training_iterator)
print(f"Number of batches in training iterator = {len(training_iterator)}")
print(f"Number of batches in validation iterator = {len(validation_iterator)}")

Creating iterators
Number of batches in training iterator = 300
Number of batches in validation iterator = 50


With the setup above DALI iterators are ready for the training. 

Next we import training utilities implemented in JAX. `init_model` will create the model instance and initialize its parameters. In this simple example it is a MLP model with two hidden layers. `update` performs one iteration of the training. `accuracy` is a helper function to run validation after each epoch on the test set and get current accuracy of the model.

In [6]:
from model import init_model, update, accuracy

At this point everything is ready to run the training.

In [7]:
print('Starting training')

model = init_model()
num_epochs = 10

for epoch in range(num_epochs):
    for batch in training_iterator:
        model = update(model, batch)

    test_acc = accuracy(model, validation_iterator)
    print(f"Epoch {epoch} sec")
    print(f"Test set accuracy {test_acc}")

Starting training
Epoch 0 sec
Test set accuracy 0.6737000346183777
Epoch 1 sec
Test set accuracy 0.7855000495910645
Epoch 2 sec
Test set accuracy 0.8254000544548035
Epoch 3 sec
Test set accuracy 0.8467000126838684
Epoch 4 sec
Test set accuracy 0.8615000247955322
Epoch 5 sec
Test set accuracy 0.8721000552177429
Epoch 6 sec
Test set accuracy 0.8778000473976135
Epoch 7 sec
Test set accuracy 0.8838000297546387
Epoch 8 sec
Test set accuracy 0.8879000544548035
Epoch 9 sec
Test set accuracy 0.8914000391960144


### Multiple GPUs

This section shows how to train the network with multiple GPUs.

In [8]:
import jax

batch_size_per_gpu = batch_size // jax.device_count()


We start by modifiying pipeline definition function. 

Note new arguments passed to the `fn.readers.caffe2`: `num_shards` and `shard_id`. They are used to controll sharding:
 - `num_shards` sets total number of shards
 - `shard_id` tells the pipeline for which shard in the training it is responsible. 

 Also `device_id` argument was removed from the decorator. Since we want these pipelines run on different GPUs we will pass particualr `device_id` in pipeline creation. Most often, `device_id` and `shard_id` will have the same value but it is not a requirement.

 If you want to learn more about DALI sharding behaviour look into [DALI sharding docs page](../../general/getting_started.ipynb).

In [9]:

@pipeline_def(batch_size=batch_size_per_gpu, num_threads=4)
def mnist_sharded_pipeline(data_path, random_shuffle, num_shards, shard_id):
    jpegs, labels = fn.readers.caffe2(
        path=data_path,
        random_shuffle=random_shuffle,
        name="mnist_caffe2_reader",
        num_shards=num_shards,
        shard_id=shard_id)
    images = fn.decoders.image(
        jpegs, device='mixed', output_type=types.GRAY)
    images = fn.crop_mirror_normalize(
        images, dtype=types.FLOAT, std=[255.], output_layout="CHW")
    images = fn.reshape(images, shape=[image_size * image_size])

    labels = labels.gpu()
    
    if random_shuffle:
        labels = fn.one_hot(labels, num_classes=num_classes)

    return images, labels

Creating pipelines. Note the `device_id` values that are passed to place a pipeline on a different device.

In [10]:
print('Creating training pipelines')

pipelines = []
for id, device in enumerate(jax.devices()):
    pipeline = mnist_sharded_pipeline(data_path=training_data_path, random_shuffle=True, num_shards=jax.device_count(), shard_id=id, device_id=id)
    print(f'Pipeline {pipeline} working on device {pipeline.device_id}')
    pipelines.append(pipeline)

print('Creating training iterator')

training_iterator = dax.DALIGenericIterator(
    pipelines,
    output_map=["images", "labels"],
    reader_name="mnist_caffe2_reader",
    auto_reset=True)

print(f"Number of batches in training iterator = {len(training_iterator)}")

Creating training pipelines
Pipeline <nvidia.dali.pipeline.Pipeline object at 0x7f2a50706d10> working on device 0
Pipeline <nvidia.dali.pipeline.Pipeline object at 0x7f2a50706d70> working on device 1
Creating training iterator
Number of batches in training iterator = 300


In [11]:
import jax.numpy as jnp


model = init_model()
model = jax.tree_map(lambda x: jnp.array([x] * jax.device_count()), model) 

In [12]:
from model import update_parallel


for epoch in range(num_epochs):
    for it, batch in enumerate(training_iterator):
        model = update_parallel(model, batch)
        
    test_acc = accuracy(jax.tree_map(lambda x: x[0], model), validation_iterator)
    
    print(f"Epoch {epoch} sec")
    print(f"Test set accuracy {test_acc}")


Epoch 0 sec
Test set accuracy 0.6892000436782837
Epoch 1 sec
Test set accuracy 0.7851000428199768
Epoch 2 sec
Test set accuracy 0.822100043296814
Epoch 3 sec
Test set accuracy 0.8433000445365906
Epoch 4 sec
Test set accuracy 0.8584000468254089
Epoch 5 sec
Test set accuracy 0.867900013923645
Epoch 6 sec
Test set accuracy 0.8770000338554382
Epoch 7 sec
Test set accuracy 0.882900059223175
Epoch 8 sec
Test set accuracy 0.8873000144958496
Epoch 9 sec
Test set accuracy 0.8915000557899475
